<a href="https://colab.research.google.com/github/ishancoderr/3D_modeling/blob/main/rasterImagePixelCount_for_landsli_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [74]:
import rasterio
import matplotlib.pyplot as plt
from numpy import asarray

# Open raster and plot
raster_Defomation = rasterio.open("/content/Deformation.tif").read(1)
raster_Aspect = rasterio.open("/content/Aspect.tif").read(1)
raster_Slope = rasterio.open("/content/Slope.tif").read(1)

In [75]:
parameter_set=[]
img_Defomation=asarray(raster_Defomation)
img_Aspect=asarray(raster_Aspect )
img_Slope=asarray(raster_Slope)
parameter_set.append(img_Slope)
parameter_set.append(img_Aspect)
parameter_set.append(img_Defomation)


In [78]:
rows = len(parameter_set[0])
cols = len(parameter_set[0][0])
print(rows,cols)

301 404


In [79]:
parameter_dict=[]
sope_dict={}
aspect_dict={}
defomation_dict={}
parameter_dict.append(sope_dict)
parameter_dict.append(aspect_dict)
parameter_dict.append(defomation_dict)
for num in range(len(parameter_dict)):
  print(num)
  rows = len(parameter_set[num])
  cols = len(parameter_set[num][0])
  for i in range(rows):
    for j in range(cols):
      if parameter_set[num][i,j] in parameter_dict[num]:
        parameter_dict[num][parameter_set[num][i,j]]+=1
      else:
        parameter_dict[num][parameter_set[num][i,j]]= 1

  sum=0
  for i,j in parameter_dict[num].items():
    if(i!=127 and i!=2147483647):
      sum=sum+j

  for i,j in parameter_dict[num].items():
    if(i!=127 and i!=2147483647):
      print(f"{i} value parameter_dict {num}:{j}/{sum}")

0
2 value parameter_dict 0:17437/18509
4 value parameter_dict 0:627/18509
1 value parameter_dict 0:445/18509
1
3 value parameter_dict 1:4210/18509
1 value parameter_dict 1:9646/18509
2 value parameter_dict 1:4653/18509
2
2 value parameter_dict 2:18495/18509
1 value parameter_dict 2:14/18509


In [81]:
# Open raster and plot
raster_Defomation_WRA = rasterio.open("/content/Aspect_WRA.tif.ovr").read(1)
raster_Aspect_WRA = rasterio.open("/content/Aspect_WRA.tif.ovr").read(1)
raster_Slope_WRA = rasterio.open("/content/Slope_WRA.tif").read(1)

/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [82]:
parameter_set=[]
img_Defomation=asarray(raster_Defomation_WRA)
img_Aspect=asarray(raster_Aspect_WRA)
img_Slope=asarray(raster_Slope_WRA)